In [37]:
# libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

import xgboost as xgb

from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

## Adaboost

* bagging
    * 다수의 표본을 추출하고 tree 모델간의 결과를 종합
    * 병렬적으로 진행되며 부트스트랩과정 포함
* boosting
    * 오차를 예측하는 모델을 생성하고 모델을 합친다.
    * 순차적으로 진행되며 부트스트랩과정이 없다.
* Adaboost
    * 시행 n 에서
        1. 시행 n-1에서 잘 분류된 데이터의 가중치를 내린다.
        2. 시행 n-1에서 잘못분류된 데이터의 가중치를 올린다.
        3. 분류한다.
        4. 분류결과 데이터를 시행 n+1로 넘긴다.
    * 모든 시행에서 나온 model에 대해 가중치를 곱하고 더한다.
    * procedure
        1. 관측값 가중치를 초기화한다.
            * $w_{i} = 1/N, i = 1, 2, ... N$ where N is number of observations
        2. for all models( m = 1, 2, .... j )
           1. 분류기 $Gm(x)$를 가중치 $w_{i}$로 적합화한다.
           2. 모델 내의 모든 관측값($x_{i}$)에 대해 손실함수를 계산한다.
               * $ L_{m} = \Sigma w_{m, i}I(y_{i} \ne G_{m}(x_{i}))$
               * $I(x)$: 조건에 따라 1, 0 을 반환. 즉 손실함수는 틀린 데이터에 대한 가중치의 합이다.
           3. $error$ 비율을 계산한다. 전체 관측값 가중치합에 대한 틀린 관측값 가중치합의 비율이다.
               * $ err_{m} = \frac{\Sigma w_{i}I(y_{i} \ne G_{m}(x_{i}))}{\Sigma w_{i}}$
           4. 모델에 대한 가중치를 계산한다.
               * $ \alpha_{m} = \frac{1-err_{m}}{err{m}} $
               * err 이 낮은 모델이 더 가중치가 크다.
           5. 데이터에 대한 가중치를 조정한다, 맞춘 관측치는 작게, 틀린 관측치는 크게(boosting) 한다.
               * $ w_{m, i} = w_{m-1, i} * \exp(\alpha_{m} * I(y_{i} \ne G_{m}(x_{i}))$
               * 모델이 결과를 맞춘경우 원래 가중치의 exp(alpha)를 곱한다.
           6. 최종함수를 만든다. 약한 모델들의 결과를 가중합한 결과가 양수면 1, 음수면 -1 을 반환한다.
               * $G(x) = sign(\Sigma \alpha_{m}G_{m}(x))$

In [18]:
root = 'C:/Users/wonca/Documents/git_repositories/TIL/2_머신러닝이론/4_tree_based_models/data/'
file_name = 'WA_Fn-UseC_-HR-Employee-Attrition.csv'
hr_df = pd.read_csv(root + file_name)

# ylabel
y = hr_df['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)
hr_df['y'] = y

# 더미변수 생성
categorical_cols = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
dummy_cols = []
for cat_col in categorical_cols:
    dummy_col = pd.get_dummies(hr_df[cat_col], prefix=cat_col)
    dummy_cols.append(dummy_col)

# 연속형 변수
origianl_numeric_cols = hr_df.select_dtypes(include=np.int64).columns.tolist()
remove_cols = ['EmployeeCount', 'EmployeeNumber', 'StandardHours']
numeric_cols = [el for el in origianl_numeric_cols if el not in remove_cols]
numeric_df = hr_df[numeric_cols]

# 더미변수 + 연속형변수
hr_new_df = pd.concat(dummy_cols + [numeric_df], axis=1)

# train test splitㅠ
x = hr_new_df.drop(['y'], axis=1)
y = hr_new_df['y']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=42)


In [19]:
# 기본 분류기로 DT 선택
dt_base_clf = DecisionTreeClassifier(criterion='gini', max_depth=1)
adaboost_clf = AdaBoostClassifier(base_estimator=dt_base_clf,
                                  n_estimators=5000,
                                  learning_rate=.05,
                                  random_state=42)

adaboost_clf.fit(x_train, y_train)


AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

In [20]:
y_train_pred = adaboost_clf.predict(x_train)
pd.crosstab(y_train, y_train_pred, colnames=['Pre'], rownames=['Act'])

Pre,0,1
Act,,
0,844,9
1,55,121


In [21]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.94      0.99      0.96       853
           1       0.93      0.69      0.79       176

    accuracy                           0.94      1029
   macro avg       0.93      0.84      0.88      1029
weighted avg       0.94      0.94      0.93      1029



In [22]:
y_test_pred = adaboost_clf.predict(x_test)
pd.crosstab(y_test, y_test_pred, colnames=['Pre'], rownames=['Act'])

Pre,0,1
Act,,
0,360,20
1,38,23


In [23]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.90      0.95      0.93       380
           1       0.53      0.38      0.44        61

    accuracy                           0.87       441
   macro avg       0.72      0.66      0.68       441
weighted avg       0.85      0.87      0.86       441



## Gradient Boosting

* 오차항을 예측하는 모델을 가중합한다.
    1. 초기관측값에 대해 모델링한다.
        * Y = F(x) + 오차항1
    2. 오차항 1에 대해 모델링하고, 설명변수를 F(x)에 추가한다.
        * 오차항1 = G(x) + 오차항2
    3. 오차항 2에 대해 모델링하고 설명변수를 G(x)에 추가한다.
        * 오차항2 = H(x) + 오차항3
    4. F, G, X를 합산(혹은 가중합산)한다.
        * Y = F(X) + G(X) + H(x) + 오차항3
* GB의 3가지 요소
    * 손실함수: 회귀문제는 rmse, 분류문제는 로그우도 사용. 각 단계에서 이전단계에서 설명하지 못한 손실에 대해 최적화를 수행한다
    * 약한분류기: 결정트리 사용
    * 손실함수를 최적화하기 위한 약한 분류기: 손실함수의 기울기가 감소하는 방향으로 새로운 트리가 하나씩 추가된다.
* procedure
    1. 

In [28]:
gb_clf = GradientBoostingClassifier(loss='deviance', 
                                    learning_rate = .05, 
                                    n_estimators= 5000, 
                                    min_samples_split= 2, 
                                    min_samples_leaf= 1, 
                                    max_depth = 1, 
                                    random_state= 42)

In [29]:
gb_clf.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=5000,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [31]:
y_train_pred = gb_clf.predict(x_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       853
           1       0.98      0.72      0.83       176

    accuracy                           0.95      1029
   macro avg       0.96      0.86      0.90      1029
weighted avg       0.95      0.95      0.95      1029



In [35]:
y_test_pred = gb_clf.predict(x_test)
print(classification_report(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93       380
           1       0.57      0.39      0.47        61

    accuracy                           0.88       441
   macro avg       0.74      0.67      0.70       441
weighted avg       0.86      0.88      0.87       441

[[362  18]
 [ 37  24]]


## xgboost

In [45]:
xgb_clf = xgb.XGBClassifier(max_depth=2, 
                            n_estimators=5000,
                            learning_rate = .02, 
                            random_state= 42)
xgb_clf.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.02, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=None, n_estimators=5000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [46]:
y_train_pred = xgb_clf.predict(x_train)
print(classification_report(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       853
           1       1.00      0.98      0.99       176

    accuracy                           1.00      1029
   macro avg       1.00      0.99      0.99      1029
weighted avg       1.00      1.00      1.00      1029

[[853   0]
 [  4 172]]


In [47]:
y_test_pred = xgb_clf.predict(x_test)
print(classification_report(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.90      0.95      0.93       380
           1       0.54      0.36      0.43        61

    accuracy                           0.87       441
   macro avg       0.72      0.66      0.68       441
weighted avg       0.85      0.87      0.86       441

[[361  19]
 [ 39  22]]
